In [1]:
import pandas as pd
import numpy as np

from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification, AdamW, BertConfig

import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('train_data.csv')
df = df[:50]
df.head()

,input,labels
0,I am reading score of Mahler is Symphony No . .,0
1,I am not interested in cars or electric applia...,1
2,This is my homework for my English class .,0
3,"In comparison , Canada is catches increased an...",0
4,"Fortunately , my older sister is friend is a d...",1


In [3]:
sentences = df.input.values
labels = df.labels.values

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [5]:
input_ids = []
attention_masks = []
for i in sentences:
    
    encoded_dict = tokenizer.encode_plus(
                        i,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        padding = True,
                        return_attention_mask = True,   # Construct attn. masks.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    
input_ids = pad_sequences(input_ids, maxlen=64, truncating="post", padding="post", dtype="int")
input_ids = np.array(input_ids)

In [6]:
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


2022-04-08 11:20:26.172438: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

In [8]:
history = model.fit(input_ids , labels)

2/2 [==============================] - 56s 8s/step - loss: 0.6917 - accuracy: 0.5000


In [9]:
model.save('SHL_BERT')

2022-04-08 11:21:48.878979: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: SHL_BERT/assets


INFO:tensorflow:Assets written to: SHL_BERT/assets
